In [1]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Create a function that returns a compiled Keras model
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(20, (5, 5), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(50, (5, 5), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Wrap the Keras model for use with TFF
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=(tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 10], dtype=tf.float32)),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()])

### Step 3: Create Federated Data
# Convert the dataset to a federated dataset
def preprocess(dataset):
    def batch_format_fn(element):
        return (tf.reshape(element['x'], [-1, 32, 32, 3]), tf.reshape(element['y'], [-1, 10]))
    
    def augment_fn(element):
        image, label = element['x'], element['y']
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, max_delta=0.1)
        return {'x': image, 'y': label}
    
    return dataset.map(augment_fn).batch(200).map(batch_format_fn)

# Create a federated dataset
NUM_CLIENTS = 10
client_data = np.array_split(x_train, NUM_CLIENTS)
client_labels = np.array_split(y_train, NUM_CLIENTS)

federated_train_data = [
    preprocess(tf.data.Dataset.from_tensor_slices({'x': client_data[i], 'y': client_labels[i]}))
    for i in range(NUM_CLIENTS)
]

### Step 4: Define the Federated Learning Process
# Create a federated averaging process
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam()
)

# Initialize the process
state = iterative_process.initialize()

eval_model = create_keras_model()
eval_model.compile(
loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()])

# Train the model for a few rounds
NUM_ROUNDS = 10
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, federated_train_data)
    
    model_weights = iterative_process.get_model_weights(state)
    model_weights.assign_weights_to(eval_model)
    eval_metrics = eval_model.evaluate(x_test, y_test, verbose=2)
    
    print(f'Round {round_num}, Metrics={metrics}, Val_Metrics={eval_metrics}')

2024-07-28 17:05:44.716025: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 17:05:44.717272: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 17:05:44.739263: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 17:05:44.739293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 17:05:44.739315: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

313/313 - 1s - loss: 2.0458 - categorical_accuracy: 0.2728 - 678ms/epoch - 2ms/step
Round 1, Metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.21112), ('loss', 2.1376772), ('num_examples', 50000), ('num_batches', 250)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))]), Val_Metrics=[2.045832633972168, 0.2727999985218048]
313/313 - 1s - loss: 1.7829 - categorical_accuracy: 0.3768 - 593ms/epoch - 2ms/step
Round 2, Metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.29736), ('loss', 1.9451816), ('num_examples', 50000), ('num_batches', 250)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))]), Val_Metrics=[1.7828880548477173, 0.376800000667572]
313/313 - 1s - loss: 1.8609 - categorical_ac